# GIL

global interpreter lock，Python 多线程另一个很重要的话题。

## 为什么有 GIL？

让我们来看看下面的代码


In [3]:
def CountDown(n):
    while n > 0:
        n -= 1

n = 100000000
%time CountDown(n)

CPU times: user 4.39 s, sys: 12.4 ms, total: 4.41 s
Wall time: 4.42 s


假设 n 是一个很大的数字 n = 100000000，耗时就是一个需要考虑的因素。

我们如何用多线程来加速？


In [4]:
from threading import Thread

def CountDown2(n):
    t1 = Thread(target=CountDown, args=[n // 2])
    t2 = Thread(target=CountDown, args=[n // 2])
    t1.start()
    t2.start()
    t1.join()
    t2.join()

n = 100000000
%time CountDown2(n)

CPU times: user 4.5 s, sys: 31.8 ms, total: 4.53 s
Wall time: 4.53 s


为什么多线程没有加速？

Python 的线程是不是假的线程？Python 的线程，的的确确封装了底层的操作系统线程，在 Linux 系统里是 Pthread（全称为 POSIX Thread），而在 Windows 系统里是 Windows Thread。另外，Python 的线程，也完全受操作系统管理，比如协调何时执行、管理内存资源、管理中断等等。

那究竟谁才是“罪魁祸首”呢？事实上，正是我们今天的主角，也就是 GIL，导致了 Python 线程的性能并不像我们期望的那样。

GIL，是最流行的 Python 解释器 CPython 中的一个技术术语。它的意思是全局解释器锁，本质上是类似操作系统的 Mutex。每一个 Python 线程，在 CPython 解释器中执行时，都会先锁住自己的线程，阻止别的线程执行。

CPython 会做一些小把戏，轮流执行 Python 线程。这样一来，用户看到的就是“伪并行”——Python 线程在交错执行，来模拟真正并行的线程。

那么，为什么 CPython 需要 GIL 呢？与 Python 的内存管理机制有关。

CPython 使用引用计数来管理内存，所有 Python 脚本中创建的实例，都会有一个引用计数，来记录有多少个指针指向它。当引用计数只有 0 时，则会自动释放内存。

举例来说，如果有两个 Python 线程同时引用了 a，就会造成引用计数的 race condition，引用计数可能最终只增加 1，这样就会造成内存被污染。因为第一个线程结束时，会把引用计数减少 1，这时可能达到条件释放内存，当第二个线程再试图访问 a 时，就找不到有效的内存了。

CPython 引进 GIL 其实主要就是这么两个原因：一是设计者为了规避类似于内存管理这样的复杂的竞争风险问题（race condition）；二是因为 CPython 大量使用 C 语言库，但大部分 C 语言库都不是原生线程安全的（线程安全会降低性能和增加复杂度）。

## GIL 是如何工作的？

![GIL work in python](https://static001.geekbang.org/resource/image/db/8d/dba8e4a107829d0b72ea513be34fe18d.png)

这张图，就是一个 GIL 在 Python 程序的工作示例。其中，Thread 1、2、3 轮流执行，每一个线程在开始执行时，都会锁住 GIL，以阻止别的线程执行；同样的，每一个线程执行完一段后，会释放 GIL，以允许别的线程开始利用资源。

为什么 Python 线程会去主动释放 GIL 呢？

- 如果 Python 线程在开始执行时锁住 GIL，而永远不去释放 GIL，那别的线程就都没有了运行的机会。

- CPython 中还有另一个机制，叫做 check_interval，意思是 CPython 解释器会去轮询检查线程 GIL 的锁住情况。每隔一段时间，Python 解释器就会强制当前线程去释放 GIL，这样别的线程才能有执行的机会。

   不同版本的 Python 中，check interval 的实现方式并不一样。早期的 Python 是 100 个 ticks，大致对应了 1000 个 bytecodes；
   
   而 Python 3 以后，interval 是 15 毫秒。
   
   我们不必细究具体多久会强制释放 GIL，这不应该成为我们程序设计的依赖条件，我们只需明白，CPython 解释器会在一个“合理”的时间范围内释放 GIL 就可以了。

整体来说，每一个 Python 线程都是类似这样循环的封装，我们来看下面这段代码：

```CPython
for (;;) {
    if (--ticker < 0) {
        ticker = check_interval;
    
        /* Give another thread a chance */
        PyThread_release_lock(interpreter_lock);
    
        /* Other threads may run now */
    
        PyThread_acquire_lock(interpreter_lock, 1);
    }

    bytecode = *next_instr++;
    switch (bytecode) {
        /* execute the next instruction ... */ 
    }
}
```

每个 Python 线程都会先检查 ticker 计数。只有在 ticker 大于 0 的情况下，线程才会去执行自己的 bytecode。

## Python 的线程安全

有了 GIL，并不意味着我们 Python 编程者就不用去考虑线程安全了。GIL 仅允许一个 Python 线程执行，但 Python 还有 check interval 这样的抢占机制。

我们来看下面一段代码：



In [42]:
import threading

n = 0
lock = threading.Lock()

def foo_unsafe():
    global n
    # 线程并不安全
    n += 1

def foo_safe():
    global n
    with lock:
        n += 1

def main(func):
    threads = []
    for i in range(100):
        t = threading.Thread(target=func)
        threads.append(t)

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print(n)

main(foo)

100


In [ ]:
用 dis 分析一下 线程不安全的方法 和 线程安全的方法

In [43]:
import dis

dis.dis(foo_unsafe)

  9           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [44]:
import dis

dis.dis(foo_safe)

 13           0 LOAD_GLOBAL              0 (lock)
              2 SETUP_WITH              14 (to 18)
              4 POP_TOP

 14           6 LOAD_GLOBAL              1 (n)
              8 LOAD_CONST               1 (1)
             10 INPLACE_ADD
             12 STORE_GLOBAL             1 (n)
             14 POP_BLOCK
             16 LOAD_CONST               0 (None)
        >>   18 WITH_CLEANUP_START
             20 WITH_CLEANUP_FINISH
             22 END_FINALLY
             24 LOAD_CONST               0 (None)
             26 RETURN_VALUE


## 如何绕过 GIL？

大部分应用情况下，你并不需要过多考虑 GIL。因为如果多线程计算成为性能瓶颈，往往已经有 Python 库来解决这个问题了。如果对性能有超严格的要求，那 Python 可能不是应该选择的语言。

记住，绕过 GIL 的大致思路有这么两种就够了：

- 绕过 CPython，使用 JPython（Java 实现的 Python 解释器）等别的实现；
- 把关键性能代码，放到别的语言（一般是 C++）中实现。

## 总结

我们不必深究一些原理的细节，明白其主要机制和存在的隐患即可。

## 思考题

- 在我们处理 cpu-bound 的任务时，为什么有时候使用多线程会比单线程还要慢些？

   答：cpu-bound 属于计算密集型程序，用多线程运行时，每个线程在开始执行时都会锁住 GIL、执行完会释放GIL，这两个步骤比较费时。相比单线程就没有切换线程的问题，所以更快。

   CPU 密集型任务加速，使用多线程是无效的，请使用多进程。这里所谓的 CPU 密集型任务，是指会消耗大量 CPU 资源的任务，比如求 1 到 100000000 的乘积，或者是把一段很长的文字编码后又解码等等。

   使用多线程之所以无效，因为 Python 多线程的本质是多个线程互相切换，但同一时刻仍然只允许一个线程运行。因此，即使使用多线程，和使用一个主线程，本质上来说并没有什么差别；反而在很多情况下，因为线程切换带来额外损耗，还会降低程序的效率。

   而如果使用多进程，就可以允许多个进程之间 in parallel 地执行任务，所以能够有效提高程序的运行效率。

   相反，在处理多阻塞高延迟的 IO 密集型程序时，因为多线程有 check interval 机制，若遇阻塞，CPython 会强制当前线程让出（释放）GIL，给其他线程执行的机会。所以能提高程序的执行效率。

   对 I/O 密集型任务，如果想要加速，请优先使用多线程或 Asyncio。当然，使用多进程也可以达到目的，但是完全没有这个必要。因为对 I/O 密集型任务来说，大多数时间都浪费在了 I/O 等待上。因此，在一个线程 / 任务等待 I/O 时，我们只需要切换线程 / 任务去执行其他 I/O 操作就可以了。

   如果 I/O 操作非常多、非常 heavy，需要建立的连接也比较多时，我们一般会选择 Asyncio。因为 Asyncio 的任务切换更加轻量化，并且它能启动的任务数也远比多线程启动的线程数要多。

- 线程颠簸 (thrashing)
